In [24]:
import pandas as pd

unique_key = ["off_last_name","off_first_name","off_middle_initial","off_rank","off_star","off_race","off_sex","off_year_of_birth","off_appointed"]
profs_df = pd.DataFrame(columns=unique_key)
intermediary_dfs = []

with pd.read_csv("../files/events/assignment_cleaned.csv", chunksize=10000) as reader:
  for chunk in reader:
    profs_df = pd.concat([profs_df, chunk.loc[:,unique_key]]).drop_duplicates(subset=unique_key)

profs_df.sort_values(by=["off_last_name", "off_first_name", "off_middle_initial"], inplace=True, ignore_index=True)
profs_df.to_csv("../files/profiles/officer_profs_test.csv")

In [94]:
import pandas as pd
import numpy as np
from functools import partial

def star_at(index: int, stars: pd.Series) -> int:
  try:
    return int(stars.dropna().sort_values().reset_index(drop=True).iat[index])
  except:
    return np.nan

profs = pd.read_csv("../files/profiles/officer_profs_test.csv")
cols = ["off_last_name","off_first_name","off_middle_initial","off_rank","off_star","off_race","off_sex","off_year_of_birth","off_appointed"]
group_key = ["off_last_name","off_first_name","off_middle_initial","off_sex","off_year_of_birth","off_appointed"]
# for the off_star_x cols to be useful we probably want to order by date desc
# otherwise this is just useful for disambiguation
profs_df = profs.groupby(group_key).agg(
  off_race=("off_race", "first"),
  ranks_held=("off_rank", lambda ranks: ', '.join(ranks.dropna().unique())),
  off_star_0=("off_star", partial(star_at, 0)),
  off_star_1=("off_star", partial(star_at, 1)),
  off_star_2=("off_star", partial(star_at, 2)),
  off_star_3=("off_star", partial(star_at, 3)),
  off_star_4=("off_star", partial(star_at, 4)),
  off_star_5=("off_star", partial(star_at, 4)),
).reset_index()

profs_df.index.name = "off_uniq_id"

profs_df.to_csv("../files/profiles/officers_index.csv")

In [92]:
print(profs_df.shape[0])
print(profs_df[profs_df["off_race"].isna()].shape[0])
profs_df.dropna(subset=["off_star_5"])
# profs_df[profs_df["off_race"].isna()].shape[0]
# profs.drop_duplicates()
# profs.pivot(index=["off_last_name","off_first_name","off_middle_initial","off_year_of_birth","off_appointed"], columns=)
# profs.groupby(["off_last_name","off_first_name","off_middle_initial","off_race","off_year_of_birth","off_appointed"])
# profs.groupby("off_star", sort=False, dropna=False).first().to_csv("../files/events/off_prof_uniq_star.csv")

12482
436


,off_last_name,off_first_name,off_middle_initial,off_sex,off_year_of_birth,off_appointed,off_race,ranks_held,off_star_0,off_star_1,off_star_2,off_star_3,off_star_4,off_star_5
3936,GOLDEN,MICHAEL,A,M,1991,2015-12-14,WHITE,POLICE OFFICER,3156.0,16833.0,18771.0,19217.0,19686.0,19686.0
10767,STEVENS,JILL,M,F,1978,2002-03-25,WHITE,COMMANDER,24.0,178.0,626.0,627.0,1222.0,1222.0
